# Download XML efetch records for PubMed IDs in the specified journals

Uses functions from `pubmedpy` rather than the `eutils` Python package, which [does not support](https://github.com/biocommons/eutils/issues/124/) pagination.

In [1]:
import pathlib
import lzma
import tqdm

from pubmedpy.eutilities import esearch_query, download_pubmed_ids

In [2]:
# NLM Title Abbreviations from https://www.ncbi.nlm.nih.gov/nlmcatalog/journals
# journals = [
#     "Bioinformatics",
#     "BMC Bioinformatics",
#     "PLoS Comput Biol",
# ]

In [3]:
%%time
# for journal in journals:
payload = {
    'db': 'pubmed',
#         'term': f'"journal article"[pt] AND 1993:2019[dp] AND "{journal}"[Journal]',
    'term': f'"journal article"[pt] AND 1993:2019[dp] AND "computational biology"[MeSH Terms] AND English[Language]',
}
pubmed_ids = esearch_query(payload, tqdm=tqdm.tqdm_notebook)
pubmed_ids = sorted(map(int, pubmed_ids))
print(f'{len(pubmed_ids):,} articles for English computational biology')


177,262 articles for English computational biology
CPU times: user 1.44 s, sys: 266 ms, total: 1.71 s
Wall time: 18.8 s


In [4]:
%%time
path = pathlib.Path('data/pubmed/efetch').joinpath('compbio-english.xml.xz')
path.parent.mkdir(parents=True, exist_ok=True)
with lzma.open(path, 'wt') as write_file:
    download_pubmed_ids(
        pubmed_ids, write_file, endpoint='efetch',
        retmax=200, retmin=50, sleep=0, error_sleep=1,
        tqdm=tqdm.tqdm_notebook,
    )


CPU times: user 20min 55s, sys: 19.7 s, total: 21min 15s
Wall time: 58min 13s
